Create a new Python application

In [23]:
pip install azure-cognitiveservices-vision-customvision


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [24]:
# Azure의 Custom Vision 라이브러리를 추가. 예측을 위하여 prediction을 포함
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
# OpenAPI 스펙에 맞춰서 Authentication을 처리할 수 있도록 해주는 코드
from msrest.authentication import ApiKeyCredentials
# Matplotlib의 pyplot을 포함하여 예측 결과를 그리기
from matplotlib import pyplot as plt
# Python Image 라이브러리로 이미지 그리기
from PIL import Image, ImageDraw, ImageFont
# Python Numpy (수학 및 과학 연산 패키지) 포함
import numpy as np
# 파일 처리 작업을 위해 os 라이브러리 포함
import os

In [25]:
# 사용자가 만든 AI 모델의 예측 기능을 사용하기 위한 endpoint 지정
prediction_endpoint = "https://southcentralus.api.cognitive.microsoft.com" #이거 수정하세요
# KEY 값 지정
prediction_key = "9d4f2bbf08f544b4a700983e66874ee4" #이거 수정하세요
# 프로젝트 ID 지정
project_id = "02a5329b-ccdf-4f91-b0b3-7345baca5c4e" #이거 수정하세요
# 모델명 지정
model_name = "Iteration5" #이거 수정하세요

In [26]:
# 앞에서 지정한 API KEY를 써서 커스텀 비전 모델을 사용할 클라이언트를 인증
credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
# endpoint를 써서 클라이언트 등록
predictor = CustomVisionPredictionClient(endpoint=prediction_endpoint, credentials=credentials)

In [27]:
import os
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

# 테스트 파일이 저장된 디렉토리 경로와 출력 파일 경로
input_dir = r"C:\Users\USER\Desktop\파이썬\data" #이거 수정하세요
output_dir = r"C:\Users\USER\Desktop\파이썬\data" #이거 수정하세요

# 출력 디렉토리가 없으면 생성
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 디렉토리 내 모든 'test-'로 시작하는 파일을 순회
for image_file in os.listdir(input_dir):
    if image_file.endswith(".jpg"):
        full_image_path = os.path.join(input_dir, image_file)
        print(f'Detecting objects in {full_image_path}')

        # Python Imaging Library의 image open함수를 써서 테스트 이미지 파일 오픈
        image = Image.open(full_image_path)

        # Numpy에서 이미지의 shape을 높이, 폭, 채널 읽기
        h, w, ch = np.array(image).shape
        print(h, w, ch)

        # 테스트 이미지를 열고 모델에 적용해서 결과를 저장
        with open(full_image_path, mode="rb") as image_data:
            results = predictor.detect_image(project_id, model_name, image_data)

        # 그래프 크기 지정하고 축 비활성화
        fig = plt.figure(figsize=(8, 8))
        plt.axis('off')

        # 개체 인식 박스를 magenta로 지정
        draw = ImageDraw.Draw(image)
        lineWidth = int(w / 100)
        color = 'magenta'

        # 예측된 객체들에 대해 확률이 50% 이상인 경우 바운딩 박스 그리기
        for prediction in results.predictions:
            if prediction.probability * 100 > 50:
                # 바운딩 박스 좌표 계산
                left = prediction.bounding_box.left * w
                top = prediction.bounding_box.top * h
                width = prediction.bounding_box.width * w
                height = prediction.bounding_box.height * h

                # 바운딩 박스를 그리기 위한 좌표 설정
                points = [(left, top), (left + width, top), (left + width, top + height), (left, top + height), (left, top)]

                # 바운딩 박스를 magenta 색으로 그림
                draw.line(points, fill=color, width=lineWidth)

                # 바운딩 박스에 태그 이름과 확률 표시
                plt.annotate(f'{prediction.tag_name} {prediction.probability * 100:.2f}%', (left, top), color=color)

        # 이미지를 Numpy 배열로 변환하여 Matplotlib에 표시
        plt.imshow(np.array(image))

        # 파일 이름에 맞춰 output 폴더에 저장
        output_file_path = os.path.join(output_dir, image_file)
        fig.savefig(output_file_path)
        print(f'Results saved in {output_file_path}')



In [28]:
import os
import xml.etree.ElementTree as ET
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry, ImageFileCreateBatch, Region
from msrest.authentication import ApiKeyCredentials
from PIL import Image

# Azure Custom Vision 설정
ENDPOINT = "https://southcentralus.api.cognitive.microsoft.com"  # Custom Vision의 엔드포인트
TRAINING_KEY = "9d4f2bbf08f544b4a700983e66874ee4"  # Custom Vision의 Training Key
PROJECT_ID = "02a5329b-ccdf-4f91-b0b3-7345baca5c4e"  # Custom Vision 프로젝트 ID

credentials = ApiKeyCredentials(in_headers={"Training-key": TRAINING_KEY})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)

# Custom Vision 프로젝트에서 'Kick-scooter' 태그 가져오기
tags = trainer.get_tags(PROJECT_ID)
kick_scooter_tag = next((t for t in tags if t.name == 'Kick-scooter'), None)

if not kick_scooter_tag:
    raise ValueError("프로젝트에 'Kick-scooter' 태그가 존재하지 않습니다. Custom Vision 프로젝트에서 태그를 생성하세요.")

# Pascal VOC XML 라벨 경로 및 이미지 경로 지정
base_path = r'C:\Users\USER\Desktop\파이썬\data'

# XML 파일 순회
image_entries = []

for label_file in os.listdir(base_path):
    if label_file.endswith('.xml'):
        tree = ET.parse(os.path.join(base_path, label_file))
        root = tree.getroot()
        
        # 이미지 파일명 가져오기
        image_filename = root.find('filename').text
        image_filepath = os.path.join(base_path, image_filename)
        
        # 이미지가 존재하는지 확인
        if os.path.exists(image_filepath):
            with Image.open(image_filepath) as img:
                img_width, img_height = img.size
            
            # 바운딩 박스 정보 추출
            regions = []
            for obj in root.findall('object'):
                tag_name = obj.find('name').text
                if tag_name.lower() == 'kickboard':  # 'kickboard'를 'Kick-scooter'로 변환
                    bndbox = obj.find('bndbox')
                    x_min = int(bndbox.find('xmin').text)
                    y_min = int(bndbox.find('ymin').text)
                    x_max = int(bndbox.find('xmax').text)
                    y_max = int(bndbox.find('ymax').text)
                    
                    # 바운딩 박스 조정
                    if x_max > img_width:
                        x_max = img_width
                    if y_max > img_height:
                        y_max = img_height
                    
                    # 유효성 검사
                    if x_min >= 0 and y_min >= 0 and x_max > x_min and y_max > y_min:
                        regions.append(Region(
                            tag_id=kick_scooter_tag.id,
                            left=x_min / img_width,
                            top=y_min / img_height,
                            width=(x_max - x_min) / img_width,
                            height=(y_max - y_min) / img_height
                        ))
                    else:
                        print(f"이미지 {image_filename}에 잘못된 바운딩 박스 발견: ({x_min}, {y_min}, {x_max}, {y_max})")

            # 이미지 엔트리 생성
            if regions:  # 유효한 바운딩 박스가 있는 경우에만 업로드
                with open(image_filepath, "rb") as image_contents:
                    image_data = image_contents.read()
                    image_entry = ImageFileCreateEntry(
                        name=image_filename,
                        contents=image_data,
                        regions=regions
                    )
                    image_entries.append(image_entry)

# Custom Vision에 이미지 업로드 (64개씩 나누어 처리)
batch_size = 64
for i in range(0, len(image_entries), batch_size):
    batch = ImageFileCreateBatch(images=image_entries[i:i + batch_size])
    upload_result = trainer.create_images_from_files(PROJECT_ID, batch)
    
    # 업로드 결과 확인
    if upload_result.is_batch_successful:
        print(f"배치 {i // batch_size + 1} 업로드 성공")
    else:
        print(f"배치 {i // batch_size + 1} 업로드 실패")
        for image in upload_result.images:
            if not image.status == 'OK':
                print(f"이미지 {image.source_url} 업로드 실패, 상태: {image.status}")

print("모든 이미지 업로드 및 태그 추가 완료")


모든 이미지 업로드 및 태그 추가 완료
